In [2]:
import pandas as pd
from sqlalchemy import create_engine
from dotenv import load_dotenv
import os
import matplotlib.pyplot as plt
import seaborn as sns ; sns.set()


load_dotenv('BDD_URL.env')
BDD_URL = os.environ['BDD_URL']
engine = create_engine(BDD_URL)

In [3]:
SQL = "SELECT * FROM principal.title_basics LIMIT 5000"
basics_df = pd.read_sql(SQL, engine)
basics_df['feature'] = basics_df['titleType'] + ' ' + basics_df['primaryTitle'] + ' ' + str(basics_df['startYear'])# + ' ' + basics_df['titleType']

engine.dispose()

basics_df

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,feature
0,tt8670526,tvEpisode,Episode #1.81,Episode #1.81,0,1973.0,None,45.0,"Drama,Romance",tvEpisode Episode #1.81 0 1973.0\n1 ...
1,tt8670528,tvEpisode,Episode #1.82,Episode #1.82,0,1973.0,None,45.0,"Drama,Romance",tvEpisode Episode #1.82 0 1973.0\n1 ...
2,tt8670530,tvEpisode,Episode #1.83,Episode #1.83,0,1973.0,None,45.0,"Drama,Romance",tvEpisode Episode #1.83 0 1973.0\n1 ...
3,tt8670532,tvEpisode,Episode #1.84,Episode #1.84,0,1973.0,None,45.0,"Drama,Romance",tvEpisode Episode #1.84 0 1973.0\n1 ...
4,tt8670534,tvEpisode,Episode #1.85,Episode #1.85,0,1973.0,None,45.0,"Drama,Romance",tvEpisode Episode #1.85 0 1973.0\n1 ...
...,...,...,...,...,...,...,...,...,...,...
4995,tt8702016,tvEpisode,Episode #1.65,Episode #1.65,0,1974.0,None,45.0,"Drama,Romance",tvEpisode Episode #1.65 0 1973.0\n1 ...
4996,tt8702018,tvSeries,Bondi Slayer,Bondi Slayer,0,2020.0,None,41.0,"Action,Comedy,Fantasy",tvSeries Bondi Slayer 0 1973.0\n1 ...
4997,tt8702020,tvEpisode,Episode #1.66,Episode #1.66,0,1974.0,None,45.0,"Drama,Romance",tvEpisode Episode #1.66 0 1973.0\n1 ...
4998,tt8702022,tvEpisode,Episode #1.67,Episode #1.67,0,1974.0,None,45.0,"Drama,Romance",tvEpisode Episode #1.67 0 1973.0\n1 ...


In [4]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

cv = CountVectorizer()

count_matrix = cv.fit_transform(basics_df['feature'])

cosine_sim = cosine_similarity(count_matrix)

cosine_sim

array([[1.        , 0.98214286, 0.98214286, ..., 0.98214286, 0.98214286,
        0.98214286],
       [0.98214286, 1.        , 0.98214286, ..., 0.98214286, 0.98214286,
        0.98214286],
       [0.98214286, 0.98214286, 1.        , ..., 0.98214286, 0.98214286,
        0.98214286],
       ...,
       [0.98214286, 0.98214286, 0.98214286, ..., 1.        , 0.98214286,
        0.98214286],
       [0.98214286, 0.98214286, 0.98214286, ..., 0.98214286, 1.        ,
        0.98214286],
       [0.98214286, 0.98214286, 0.98214286, ..., 0.98214286, 0.98214286,
        1.        ]])

In [6]:
def findfilm(index):
    return basics_df.iloc[index]['primaryTitle']

def getindex(filmm):
    return basics_df[basics_df['primaryTitle'] == filmm].index[0]

movieee = 'Bondi Slayer'
getindex(movieee)

4996

In [8]:
def transformationliste(array):
    d = list(enumerate(array, 0))
    return d

def recommend(matrice=cosine_sim, film='Table Dhoe', df=basics_df, proposition=5):
    indexfilm = getindex(film)
    vecteursimilarite = matrice[indexfilm]
    liste = transformationliste(vecteursimilarite)
    return vecteursimilarite


recommend(film='Bondi Slayer')

0.9464285714285721

In [9]:
def recommend(filmmm):
    # Trouve l'indice du film dans le DataFrame basics_df où le titre correspond à filmmm
    indexfilm = basics_df[basics_df['primaryTitle'] == filmmm].index[0]
    print(indexfilm)
    # Enumère les scores de similarité pour le film spécifié dans la matrice cosine_sim
    scoresimilarity = list(enumerate(cosine_sim[indexfilm]))
    print(type(scoresimilarity))
    # Trie les scores de similarité dans l'ordre décroissant
    scoresimilarity = sorted(scoresimilarity, key=lambda x: x[1], reverse=True)
    print(scoresimilarity)
    # Supprime le premier élément, car il s'agit du film lui-même (score de similarité maximal avec lui-même)
    scoresimilarity = scoresimilarity[1:]
    print(scoresimilarity)
    # Sélectionne les cinq premiers films avec les scores de similarité les plus élevés
    topfilms = scoresimilarity[:5]
    print(scoresimilarity)
    # Pour chaque film sélectionné, trouve son titre et ajoute-le à la liste de recommandations
    rekomand = [findfilm(movie[0]) for movie in topfilms]
    print(rekomand)
    # Retourne la liste des recommandations
    return rekomand

recommendations = recommend("Bondi Slayer")
print(recommendations)



4996
<class 'list'>
[(4996, 1.0000000000000007), (1572, 0.9730124597112042), (1611, 0.9730124597112042), (2574, 0.9730124597112042), (3875, 0.9730124597112042), (4496, 0.9730124597112042), (251, 0.964285714285715), (306, 0.964285714285715), (309, 0.964285714285715), (371, 0.964285714285715), (431, 0.964285714285715), (843, 0.964285714285715), (1216, 0.964285714285715), (1458, 0.964285714285715), (1587, 0.964285714285715), (1590, 0.964285714285715), (1822, 0.964285714285715), (1999, 0.964285714285715), (2066, 0.964285714285715), (2109, 0.964285714285715), (2167, 0.964285714285715), (2389, 0.964285714285715), (2545, 0.964285714285715), (2564, 0.964285714285715), (2566, 0.964285714285715), (2572, 0.964285714285715), (2788, 0.964285714285715), (3242, 0.964285714285715), (3338, 0.964285714285715), (3523, 0.964285714285715), (3865, 0.964285714285715), (4386, 0.964285714285715), (4439, 0.964285714285715), (1072, 0.9637956818756332), (2180, 0.9637956818756332), (3273, 0.9637956818756332), (479

In [ ]:
#plt.figure(figsize=(10, 8))
#sns.heatmap(cosine_sim, annot=True, fmt=".2f", cmap='coolwarm', center=0)
#plt.title('Matrice de simmilarité')
#plt.show()

In [ ]:
""" import numpy as np

# creating a one-dimensional numpy array
array = np.array(['a', 'b', 'c', 'd', 'e', 'f'])

# convert numpy array to dictionary
d = dict(enumerate(array, 0))

# print numpy array
print(array)
print(type(array))

# print dictionary
print(d)
print(type(d)) """

['a' 'b' 'c' 'd' 'e' 'f']
<class 'numpy.ndarray'>
{0: 'a', 1: 'b', 2: 'c', 3: 'd', 4: 'e', 5: 'f'}
<class 'dict'>
